# Intro notebook
The purpose of this notebook is to connect to a local database that we created by loading CSV files into that database.
Copy / paste this notebook into other places to add / delete cells and add new analysis.

In [26]:
import os
print(os.getcwd())

/Users/beoconno/Documents/ff/ff_data/notebooks/nfl


In [27]:
from pathlib import Path

In [180]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [182]:
# Define your database file path (SQLite example)
rel_db_path = Path.cwd().parent.parent / 'db_output' / 'nfl.db'
db_file_path = rel_db_path.resolve()
db_string = f"sqlite:///{db_file_path}"
%sql $db_string

In [283]:
%%sql
SELECT
    *,
    ROW_NUMBER() OVER(PARTITION BY team_id, position ORDER BY fppg_ppr DESC) AS pos_rank,
    CASE
        WHEN position IN ("QB", "TE") AND
            ROW_NUMBER() OVER(PARTITION BY team_id, position ORDER BY fppg_ppr DESC) = 1
            THEN fppg_ppr
        WHEN position IN ("RB", "WR") AND
            ROW_NUMBER() OVER(PARTITION BY team_id, position ORDER BY fppg_ppr DESC) <= 2
            THEN fppg_ppr
        WHEN position IN ("TE", "RB", "WR") AND
            ROW_NUMBER() OVER(PARTITION BY team_id, position ORDER BY fppg_ppr DESC) > 2
            THEN fppg_ppr
        ELSE 0
        END AS team_score
FROM draft_combos_dr10
WHERE 1=1
    AND team_id = 3
LIMIT 10

 * sqlite:////Users/beoconno/Documents/ff/ff_data/db_output/nfl.db
   sqlite:////Users/beoconno/Documents/ff/ff_data/raw_data/nfl/src_code/nfl.db
Done.


name,position,games,fppg_ppr,round,team_id,pos_rank,team_score
josh allen,QB,17,23.09,3,3,1,23.09
lamar jackson,QB,16,20.7,4,3,2,0
patrick mahomes,QB,16,17.51,2,3,3,0
bijan robinson,RB,17,14.49,1,3,1,14.49
jk dobbins,RB,1,11.7,5,3,2,11.7


In [273]:
%%sql
SELECT
    c_year AS year,
    c_pos AS position,
    CASE
        WHEN adp_pos_rank <= 3 THEN '1-3'
        WHEN adp_pos_rank <= 10 THEN '4-10'
        ELSE
            CAST((((adp_pos_rank - 1) / 10) * 10 + 1) AS VARCHAR)
            || '-' ||
            CAST((((adp_pos_rank - 1) / 10) * 10 + 10) AS VARCHAR)
        END AS adp_bucket,
    MIN(adp_pos_rank) AS bucket_start,
    COUNT(1) AS player_cnt,
    ROUND(AVG(exp_fppg_ppr),2) AS avg_expected_ppr,
    ROUND(AVG(fppg_ppr),2) AS avg_actual_ppr,
    ROUND(AVG(pick_accuracy),2) AS avg_accuracy,
    MAX(pick_accuracy) AS biggest_surprise,
    MIN(pick_accuracy) AS worst_miss
FROM nfl_adp_hype
WHERE 1=1
    AND fppg_ppr IS NOT NULL
    AND adp_pos_rank IS NOT NULL
    AND exp_fppg_ppr IS NOT NULL
GROUP BY 1, 2, 3
ORDER BY adp_pos_rank
LIMIT 20

 * sqlite:////Users/beoconno/Documents/ff/ff_data/db_output/nfl.db
   sqlite:////Users/beoconno/Documents/ff/ff_data/raw_data/nfl/src_code/nfl.db
Done.


year,position,adp_bucket,bucket_start,player_cnt,avg_expected_ppr,avg_actual_ppr,avg_accuracy,biggest_surprise,worst_miss
2016,QB,1-3,1,3,23.32,19.19,-4.12,1.13,-8.14
2016,RB,1-3,1,3,23.33,19.86,-3.48,2.66,-13.7
2016,WR,1-3,1,3,21.48,19.2,-2.28,-1.49,-2.71
2017,QB,1-3,1,3,23.32,17.8,-5.51,-4.13,-6.58
2017,RB,1-3,1,3,23.33,17.42,-5.92,-0.06,-13.09
2018,QB,1-3,1,3,23.32,19.28,-4.03,-1.89,-5.55
2018,TE,1-3,1,3,15.68,15.34,-0.34,3.12,-6.96
2018,WR,1-3,1,3,21.48,20.54,-0.94,-0.37,-1.62
2019,RB,1-3,1,3,23.33,21.99,-1.34,8.37,-7.31
2019,TE,1-3,1,3,15.68,15.38,-0.3,0.3,-1.16


In [206]:
%%sql
SELECT
    c_year AS year,
    name || " (" 
        || c_pos || adp_pos_rank || " -> " ||
        c_pos || actual_pos_fin_fppg_ppr || ")" AS label,
    c_pos AS position,
    adp_pos_rank AS draft_rank,
    exp_fppg_ppr AS expected_ppg,
    fppg_ppr AS actual_ppg,
    pick_accuracy,
    CASE
        WHEN adp_pos_rank = actual_pos_fin_fppg_ppr THEN "lightgrey"
        WHEN ABS(pick_accuracy) < 2 THEN "lightgrey"
        WHEN CAST(adp_pos_rank AS INT) > CAST(actual_pos_fin_fppg_ppr AS INT) THEN "lightgreen"
        WHEN CAST(adp_pos_rank AS INT) < CAST(actual_pos_fin_fppg_ppr AS INT) THEN "lightcoral"
        ELSE "untagged"
        END AS accuracy_color
FROM nfl_adp_hype
WHERE 1=1
    AND c_pos IN ("QB", "WR", "RB", "TE")
    AND name LIKE 'antonio brown'
ORDER BY c_year ASC
LIMIT 10

 * sqlite:////Users/beoconno/Documents/ff/ff_data/db_output/nfl.db
   sqlite:////Users/beoconno/Documents/ff/ff_data/raw_data/nfl/src_code/nfl.db
Done.


year,label,position,draft_rank,expected_ppg,actual_ppg,pick_accuracy,accuracy_color
2016,antonio brown (WR1 -> WR1),WR,1,23.2,20.49,-2.71,lightgrey
2017,antonio brown (WR1 -> WR1),WR,1,23.2,22.16,-1.04,lightgrey
2018,antonio brown (WR1 -> WR2),WR,1,23.2,21.58,-1.62,lightgrey
2019,antonio brown (WR9 -> WR9),WR,9,16.82,16.1,-0.72,lightgrey
2020,antonio brown (WR71 -> WR23),WR,71,8.16,14.64,6.48,lightcoral
2021,antonio brown (WR29 -> WR8),WR,29,13.13,17.3,4.17,lightcoral


In [250]:
%%sql
SELECT
    year,
    position,
    label
FROM image_ep_1_accuracy_graph
WHERE 1=1
    AND LABEL != ''
LIMIT 10

 * sqlite:////Users/beoconno/Documents/ff/ff_data/db_output/nfl.db
   sqlite:////Users/beoconno/Documents/ff/ff_data/raw_data/nfl/src_code/nfl.db
Done.


year,position,label
2016,QB,cam newton<br>QB1 to 13 (-8.14) ppg
2016,QB,matt ryan<br>QB19 to 2 (5.66) ppg
2016,QB,tony romo<br>QB28 to 48 (-8.64) ppg
2016,QB,mark sanchez<br>QB39 to 68 (-10.7) ppg
2016,QB,matt moore<br>QB58 to 27 (11.11) ppg
2016,QB,jacoby brissett<br>QB78 to 38 (9.58) ppg
2016,RB,todd gurley<br>RB1 to 24 (-13.7) ppg
2016,RB,adrian peterson<br>RB4 to 95 (-16.94) ppg
2016,RB,jamaal charles<br>RB8 to 78 (-13.05) ppg
2016,RB,jordan howard<br>RB62 to 9 (8.98) ppg


In [80]:
%%sql
SELECT
    adp_team,
    c_team,
    CASE
        WHEN team != team_year_prior AND team LIKE '%TM' THEN 'In season trade'
        WHEN team != team_year_prior THEN 'New team - ffy'
        ELSE ''
        END AS team_change,
    team_year_prior,
    team_next_year,
    name,
    c_year,
    yid
FROM nfl
WHERE NAME LIKE '%antonio brown%'
    OR NAME LIKE '%zack moss%'
    OR NAME LIKE '%ian mccaffrey%'
ORDER BY name, c_year ASC 

 * sqlite:////Users/beoconno/Documents/ff/ff_data/raw_data/nfl/src_code/nfl.db
Done.


adp_team,c_team,team_change,team_year_prior,team_next_year,name,c_year,yid
None,PIT,,None,PIT,antonio brown,2016,1
None,PIT,,PIT,PIT,antonio brown,2017,2
None,PIT,,PIT,NWE,antonio brown,2018,3
None,NWE,New team,PIT,TAM,antonio brown,2019,4
None,TAM,New team,NWE,TAM,antonio brown,2020,5
TB,TAM,,TAM,None,antonio brown,2021,6
SF,CAR,,None,CAR,christian mccaffrey,2017,1
SF,CAR,,CAR,CAR,christian mccaffrey,2018,2
SF,CAR,,CAR,CAR,christian mccaffrey,2019,3
SF,CAR,,CAR,CAR,christian mccaffrey,2020,4


In [56]:
%%sql
SELECT
    *
FROM hist_adp_perf
LIMIT 10

 * sqlite:////Users/beoconno/Documents/ff/ff_data/raw_data/nfl/src_code/nfl.db
Done.


actual_pos_fin_fppg_ppr,fantasy_pos,pos_w_fppg_ppr_finish,exp_fppg_ppr,n_in_exp_fppg_ppr
1,QB,QB1,25.09,8
1,RB,RB1,26.09,8
1,TE,TE1,17.05,8
1,WR,WR1,23.2,8
2,QB,QB2,22.62,8
2,RB,RB2,22.83,8
2,TE,TE2,15.59,8
2,WR,WR2,21.21,8
3,QB,QB3,22.24,8
3,RB,RB3,21.08,8


In [90]:
%%sql
SELECT
    name,
    adp_ovr,
    c_year,
    adp_pos,
    exp_fppg_ppr,
    prior_fppg_ppr,
    pick_hype,
    fppg_ppr,
    pick_accuracy,
    team_change,
    team_year_prior AS from_team,
    team AS to_team
    
FROM nfl_adp_hype
WHERE 1=1
    AND team_change LIKE 'New%'
    AND adp_ovr <= 30
    AND fantasy_pos IN ('QB', 'WR', 'RB')
ORDER BY c_year DESC, adp_ovr ASC
LIMIT 20

 * sqlite:////Users/beoconno/Documents/ff/ff_data/raw_data/nfl/src_code/nfl.db
Done.


name,adp_ovr,c_year,adp_pos,exp_fppg_ppr,prior_fppg_ppr,pick_hype,fppg_ppr,pick_accuracy,team_change,from_team,to_team
christian mccaffrey,2.0,2023,RB1,26.09,20.96,5.13,24.46,-1.63,New team - ffy,2TM,SFO
davante adams,11.0,2022,WR4,19.28,21.52,-2.24,19.74,0.46,New team - ffy,GNB,LVR
tyreek hill,20.0,2022,WR7,17.43,17.44,-0.01,20.42,2.99,New team - ffy,KAN,MIA
aj brown,30.0,2022,WR11,16.34,13.92,2.42,17.62,1.28,New team - ffy,TEN,PHI
josh jacobs,13.0,2020,RB10,16.13,14.74,1.39,15.42,-0.71,New team - ffy,OAK,LVR
kenyan drake,15.0,2020,RB11,15.55,15.3,0.25,12.81,-2.74,New team - ffy,2TM,ARI
deandre hopkins,16.0,2020,WR5,18.64,17.9,0.74,17.99,-0.65,New team - ffy,HOU,ARI
todd gurley,30.0,2020,RB16,14.39,14.63,-0.24,10.88,-3.51,New team - ffy,LAR,ATL
odell beckham,13.0,2019,WR5,18.64,19.19,-0.55,12.58,-6.06,New team - ffy,NYG,CLE
antonio brown,22.0,2019,WR9,16.82,21.58,-4.76,16.1,-0.72,New team - ffy,PIT,NWE


In [112]:
%%sql
SELECT *
FROM nfl_adp_hype
LIMIT 20

 * sqlite:////Users/beoconno/Documents/ff/ff_data/raw_data/nfl/src_code/nfl.db
Done.


adp_player,adp_ovr,adp_team,adp_pos,adp_pos_only,adp_pos_rank,adp_year,adp_avg,dr10_pick,dr10,adp_join_name,ranker,player,team,fantasy_pos,age,g,gs,pass_cmp,pass_att,pass_yds,pass_td,pass_int,rush_att,rush_yds,rush_yds_per_att,rush_td,targets,rec,rec_yds,rec_yds_per_rec,rec_td,fumbles,fumbles_lost,all_td,two_pt_md,two_pt_pass,fantasy_points,fantasy_points_ppr,draftkings_points,fanduel_points,vbd,fantasy_rank_pos,fantasy_rank_overall,player_id,Year,cleaned_player,pro_bowl,all_pro,fppg,fppgs,fppg_ppr,fantasy_points_hppr,pgp,pass_att_pg,rush_att_pg,targets_pg,pass_cmp_pg,pass_yds_pg,pass_td_pg,pass_td_rate,pass_att_ptd,rush_td_pg,rush_yds_pg,rush_yds_per_td,rush_att_per_td,rec_pg,catch_rate,rec_td_pg,rec_yds_pg,tar_per_td,rec_per_td,rec_yds_per_td,nfl_join_name,name,c_player,c_year,c_team,c_pos,fyd,lyd,yid,next_fppg,next_fppg_ppr,prior_fppg,prior_fppg_ppr,team_year_prior,team_next_year,actual_pos_fin_fppg_ppr,actual_pos_fin_fppg_std,actual_pos_fin_tot_ppr,actual_pos_fin_tot_std,n_in_exp_fppg_ppr,exp_fppg_ppr,pick_hype,pick_accuracy,team_change
Aaron Burbridge,473.0,None,WR170,WR,170,2016,440.0,3.0,48,aaron burbridge,422,Aaron Burbridge,SFO,WR,23,16,3,0,0,0,0,0,0,0,None,0,16,7,88,12.57,0,1.0,1,0,None,None,7.0,13.8,14.8,10.3,None,171,None,BurbAa00,2016,Aaron Burbridge,false,false,0.44,2.33,0.86,10.5,1.0,0.0,0.0,1.0,0.0,0.0,0.0,None,None,0.0,0.0,None,None,0.438,0.438,0.0,5.5,None,None,None,aaron burbridge,aaron burbridge,Aaron Burbridge,2016,SFO,WR,2016,2016,1,None,None,0,0,None,None,183,188,165,169,8,1.58,1.58,-0.72,
Aaron Dobson,483.0,None,WR175,WR,175,2016,450.0,3.0,49,aaron dobson,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,aaron dobson,Aaron Dobson,2016,None,WR,None,None,1,None,None,0,0,None,None,43,43,43,43,8,1.37,1.37,0,
None,None,None,None,None,None,None,None,None,None,None,617,Aaron Fuller,SEA,WR,24,1,0,0,0,0,0,0,0,0,None,0,0,0,0,None,0,0.0,0,0,None,None,None,None,None,None,None,245,None,FullAa01,2021,Aaron Fuller,false,false,None,None,None,None,0.06,0.0,0.0,0.0,0.0,0.0,0.0,None,None,0.0,0.0,None,None,0.0,None,0.0,0.0,None,None,None,aaron fuller,aaron fuller,Aaron Fuller,2021,SEA,WR,2021,2021,1,None,None,0,0,None,None,251,251,251,251,None,None,None,0,
None,None,None,None,None,None,None,None,None,None,None,544,Aaron Green,LAR,RB,24,2,0,0,0,0,0,0,0,0,None,0,0,0,0,None,0,None,0,0,None,None,None,None,None,None,None,161,None,GreeAa01,2016,Aaron Green,false,false,None,None,None,None,0.13,0.0,0.0,0.0,0.0,0.0,0.0,None,None,0.0,0.0,None,None,0.0,None,0.0,0.0,None,None,None,aaron green,aaron green,Aaron Green,2016,LAR,RB,2016,2016,1,None,None,0,0,None,None,158,158,158,155,None,None,None,0,
Aaron Hernandez,385.0,None,TE44,TE,44,2017,344.0,5.0,39,aaron hernandez,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,aaron hernandez,Aaron Hernandez,2017,None,TE,None,None,1,None,None,0,0,None,None,95,95,95,95,8,3.97,3.97,0,
Aaron Hernandez,413.0,None,TE58,TE,58,2018,400.0,3.0,42,aaron hernandez,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,aaron hernandez,Aaron Hernandez,2018,None,TE,None,None,2,None,None,0,0,None,None,128,128,128,128,8,2.77,2.77,0,
Aaron Hernandez,416.0,None,TE46,TE,46,2019,418.0,6.0,42,aaron hernandez,None,None,None,None,None,None,Non

In [133]:
%%sql
PRAGMA table_info(nfl_adp_hype);

 * sqlite:////Users/beoconno/Documents/ff/ff_data/raw_data/nfl/src_code/nfl.db
Done.


cid,name,type,notnull,dflt_value,pk
0,adp_player,TEXT,0,None,0
1,adp_ovr,REAL,0,None,0
2,adp_team,TEXT,0,None,0
3,adp_pos,TEXT,0,None,0
4,adp_pos_only,,0,None,0
5,adp_pos_rank,,0,None,0
6,adp_year,TEXT,0,None,0
7,adp_avg,REAL,0,None,0
8,dr10_pick,,0,None,0
9,dr10,,0,None,0


In [155]:
%%sql
SELECT
    IIF(c_team LIKE '%TM',team_year_prior,c_team) AS fill_team,
    c_year,
    c_pos,
    COUNT(1) AS players_drafted,
    MIN((IIF(depth_rank_adp=1,adp_ovr,NULL))) AS adp_ovr_depth1,
MIN((IIF(depth_rank_adp=2,adp_ovr,NULL))) AS adp_ovr_depth2,
MIN((IIF(depth_rank_adp=3,adp_ovr,NULL))) AS adp_ovr_depth3,

MIN((IIF(depth_rank_adp=1,age,NULL))) AS age_depth1,
MIN((IIF(depth_rank_adp=2,age,NULL))) AS age_depth2,
MIN((IIF(depth_rank_adp=3,age,NULL))) AS age_depth3,

MIN((IIF(depth_rank_adp=1,g,NULL))) AS g_depth1,
MIN((IIF(depth_rank_adp=2,g,NULL))) AS g_depth2,
MIN((IIF(depth_rank_adp=3,g,NULL))) AS g_depth3,

MIN((IIF(depth_rank_adp=1,adp_pos_rank,NULL))) AS adp_pos_rank_d1,
MIN((IIF(depth_rank_adp=2,adp_pos_rank,NULL))) AS adp_pos_rank_d2,
MIN((IIF(depth_rank_adp=3,adp_pos_rank,NULL))) AS adp_pos_rank_d3
FROM nfl_adp_hype
WHERE
    c_team IS NOT NULL
    AND c_pos = 'RB'
    AND c_team = 'LAR'
GROUP BY 1, 2, 3
ORDER BY 2 DESC, 1 ASC
LIMIT 30

 * sqlite:////Users/beoconno/Documents/ff/ff_data/raw_data/nfl/src_code/nfl.db
Done.


fill_team,c_year,c_pos,players_drafted,adp_ovr_depth1,adp_ovr_depth2,adp_ovr_depth3,age_depth1,age_depth2,age_depth3,g_depth1,g_depth2,g_depth3,adp_pos_rank_d1,adp_pos_rank_d2,adp_pos_rank_d3
LAR,2024,RB,2,20.0,115.0,None,None,None,None,None,None,None,7,36,None
LAR,2023,RB,5,257.0,274.0,None,23,22,None,12,10,None,71,73,None
LAR,2022,RB,5,43.0,116.0,None,23,25,None,15,10,None,20,40,None
LAR,2021,RB,5,45.0,97.0,None,24,26,None,12,17,None,22,39,None
LAR,2020,RB,5,60.0,124.0,245.0,21,23,27,13,15,16,25,45,73
LAR,2019,RB,4,14.0,120.0,190.0,25,22,26,15,13,14,9,43,60
LAR,2018,RB,4,1.0,217.0,297.0,24,22,25,14,4,12,1,67,80
LAR,2017,RB,4,18.0,395.0,None,23,24,None,15,11,None,10,114,None
LAR,2016,RB,6,4.0,256.0,None,22,26,None,16,11,None,1,80,None


In [153]:
%%sql
SELECT
    IIF(c_team LIKE '%TM',team_year_prior,c_team) AS fill_team,
    c_year,
    c_pos,
    name,
    adp_ovr,
    fantasy_points_ppr,
    depth_rank_tot,
    fppg_ppr,
    depth_rank_ppg_ppr,
    depth_rank_adp
FROM nfl_adp_hype
WHERE 1=1
    AND c_pos = 'RB'
    AND C_team = 'LAR'
ORDER BY fppg_ppr DESC 
LIMIT 10

 * sqlite:////Users/beoconno/Documents/ff/ff_data/raw_data/nfl/src_code/nfl.db
Done.


fill_team,c_year,c_pos,name,adp_ovr,fantasy_points_ppr,depth_rank_tot,fppg_ppr,depth_rank_ppg_ppr,depth_rank_adp
LAR,2018,RB,todd gurley,1.0,372.1,1,26.58,1,1
LAR,2017,RB,todd gurley,18.0,383.3,1,25.55,1,1
LAR,2023,RB,kyren williams,257.0,255.0,1,21.25,1,1
LAR,2019,RB,todd gurley,14.0,219.4,1,14.63,1,1
LAR,2021,RB,darrell henderson,45.0,163.4,1,13.62,1,1
LAR,2016,RB,todd gurley,4.0,198.2,1,12.39,1,1
LAR,2023,RB,darrell henderson,None,43.5,3,10.88,2,None
LAR,2022,RB,cam akers,43.0,141.3,1,9.42,1,1
LAR,2020,RB,darrell henderson,124.0,130.3,1,8.69,1,2
LAR,2021,RB,sony michel,97.0,146.3,2,8.61,2,2


In [166]:
%%sql
SELECT * 
FROM nfl_window_stats
limit 10

 * sqlite:////Users/beoconno/Documents/ff/ff_data/raw_data/nfl/src_code/nfl.db
Done.


adp_player,adp_ovr,adp_team,adp_pos,adp_pos_only,adp_pos_rank,adp_year,adp_avg,dr10_pick,dr10,adp_join_name,ranker,player,team,fantasy_pos,age,g,gs,pass_cmp,pass_att,pass_yds,pass_td,pass_int,rush_att,rush_yds,rush_yds_per_att,rush_td,targets,rec,rec_yds,rec_yds_per_rec,rec_td,fumbles,fumbles_lost,all_td,two_pt_md,two_pt_pass,fantasy_points,fantasy_points_ppr,draftkings_points,fanduel_points,vbd,fantasy_rank_pos,fantasy_rank_overall,player_id,Year,cleaned_player,pro_bowl,all_pro,fppg,fppgs,fppg_ppr,fantasy_points_hppr,pgp,pass_att_pg,rush_att_pg,targets_pg,pass_cmp_pg,pass_yds_pg,pass_td_pg,pass_td_rate,pass_att_ptd,rush_td_pg,rush_yds_pg,rush_yds_per_td,rush_att_per_td,rec_pg,catch_rate,rec_td_pg,rec_yds_pg,tar_per_td,rec_per_td,rec_yds_per_td,nfl_join_name,name,c_player,c_year,c_team,c_pos,fyd,lyd,yid,next_fppg,next_fppg_ppr,prior_fppg,prior_fppg_ppr,team_year_prior,team_next_year,fill_team,actual_pos_fin_fppg_ppr,actual_pos_fin_fppg_std,actual_pos_fin_tot_ppr,actual_pos_fin_tot_std
Aaron Burbridge,473.0,None,WR170,WR,170,2016,440.0,3.0,48,aaron burbridge,422,Aaron Burbridge,SFO,WR,23,16,3,0,0,0,0,0,0,0,None,0,16,7,88,12.57,0,1.0,1,0,None,None,7.0,13.8,14.8,10.3,None,171,None,BurbAa00,2016,Aaron Burbridge,false,false,0.44,2.33,0.86,10.5,1.0,0.0,0.0,1.0,0.0,0.0,0.0,None,None,0.0,0.0,None,None,0.438,0.438,0.0,5.5,None,None,None,aaron burbridge,aaron burbridge,Aaron Burbridge,2016,SFO,WR,2016,2016,1,None,None,0,0,None,None,SFO,183,188,165,169
Aaron Dobson,483.0,None,WR175,WR,175,2016,450.0,3.0,49,aaron dobson,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,aaron dobson,Aaron Dobson,2016,None,WR,None,None,1,None,None,0,0,None,None,None,43,43,43,43
None,None,None,None,None,None,None,None,None,None,None,617,Aaron Fuller,SEA,WR,24,1,0,0,0,0,0,0,0,0,None,0,0,0,0,None,0,0.0,0,0,None,None,None,None,None,None,None,245,None,FullAa01,2021,Aaron Fuller,false,false,None,None,None,None,0.06,0.0,0.0,0.0,0.0,0.0,0.0,None,None,0.0,0.0,None,None,0.0,None,0.0,0.0,None,None,None,aaron fuller,aaron fuller,Aaron Fuller,2021,SEA,WR,2021,2021,1,None,None,0,0,None,None,SEA,251,251,251,251
None,None,None,None,None,None,None,None,None,None,None,544,Aaron Green,LAR,RB,24,2,0,0,0,0,0,0,0,0,None,0,0,0,0,None,0,None,0,0,None,None,None,None,None,None,None,161,None,GreeAa01,2016,Aaron Green,false,false,None,None,None,None,0.13,0.0,0.0,0.0,0.0,0.0,0.0,None,None,0.0,0.0,None,None,0.0,None,0.0,0.0,None,None,None,aaron green,aaron green,Aaron Green,2016,LAR,RB,2016,2016,1,None,None,0,0,None,None,LAR,158,158,158,155
Aaron Hernandez,385.0,None,TE44,TE,44,2017,344.0,5.0,39,aaron hernandez,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,aaron hernandez,Aaron Hernandez,2017,None,TE,None,None,1,None,None,0,0,None,None,None,95,95,95,95
Aaron Hernandez,413.0,None,TE58,TE,58,2018,400.0,3.0,42,aaron hernandez,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,aaron hernandez,Aaron Hernandez,2018,None,TE,None,None,2,None,None,0,0,None,None,None,128,128,128,128
Aaron Hernandez,416.0,None,TE46,TE,46,2019,418.0,6.0,42,aaron hernandez,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Non

In [175]:
%%sql
SELECT * 
FROM nfl_position_groups
WHERE TEAM IS NOT NULL AND TEAM != '2TM' AND TEAM = 'ARI' AND c_pos != 'TE'
order by c_pos,c_year DESC
limit 30

 * sqlite:////Users/beoconno/Documents/ff/ff_data/raw_data/nfl/src_code/nfl.db
Done.


team,c_year,c_pos,adp_ovr_depth1,adp_ovr_depth2,adp_ovr_depth3,age_depth1,age_depth2,age_depth3,g_depth1,g_depth2,g_depth3,adp_pos_rank_d1,adp_pos_rank_d2,adp_pos_rank_d3,yid_d1,yid_d2,yid_d3,tot_pass_cmp,tot_pass_att,tot_pass_yds,tot_pass_td,tot_pass_int,tot_rush_att,tot_rush_yds,tot_rush_td,tot_targets,tot_rec,tot_rec_yds,tot_rec_td,tot_fumbles,tot_fumbles_lost,tot_all_td,tot_fantasy_points_ppr,tot_fppg_ppr,tot_exp_fppg_ppr,tot_pick_hype,tot_pick_accuracy,cnt_team_change
ARI,2018,FB,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0,0,0,0,0,2,2,0,7,7,37,0,0.0,0,0,10.9,1.09,None,None,0,1
ARI,2021,PK,178.0,None,None,None,None,None,None,None,None,9,None,None,1,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0,1
ARI,2024,QB,71.0,None,None,None,None,None,None,None,None,8,None,None,6,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,19.28,0.98,0,1
ARI,2023,QB,203.0,273.0,None,26,24,None,8,7,None,29,36,None,5,1,None,188,289,1861,10,7,52,274,4,0,0,0,0,9.0,1,4,153.9,19.37,24.82,6.59,-5.449999999999999,2
ARI,2022,QB,55.0,None,None,25,None,None,11,None,None,5,None,None,4,None,None,432,663,3962,17,17,100,520,3,0,0,0,0,13.0,5,3,260.5,37.160000000000004,20.84,-0.62,-2.61,4
ARI,2021,QB,42.0,None,None,24,None,None,14,None,None,3,None,None,3,None,None,413,589,4563,27,11,113,466,5,0,0,7,0,18.0,1,5,343.8,27.62,22.24,-1.43,-0.78,3
ARI,2020,QB,57.0,None,None,23,None,None,16,None,None,5,None,None,2,None,None,386,574,4076,27,13,137,834,11,0,0,0,0,10.0,4,11,386.4,25.21,20.84,3.01,2.83,2
ARI,2019,QB,97.0,None,None,22,None,None,16,None,None,11,None,None,1,None,None,354,553,3771,20,12,100,585,4,0,0,0,0,5.0,2,4,291.40000000000003,19.86,18.54,18.54,-0.71,2
ARI,2018,QB,256.0,313.0,None,31,21,None,3,14,None,31,37,None,3,1,None,282,494,2852,14,18,25,145,0,0,0,0,0,13.0,7,0,136.60000000000002,17.79,24.2,10.7,-11.91,3
ARI,2017,QB,136.0,None,None,38,None,None,7,None,None,18,None,None,2,None,None,338,597,3958,21,18,45,101,0,0,0,0,0,12.0,2,0,214.4,37.03,16.31,0.1,-1.7,3


In [164]:
%%sql
SELECT
    fill_team AS team,
    c_year,
    c_pos,
    -- OVERALL ADP FOR THE PLAYER IN DEPTH 1, 2, 3 OF EACH POSITION GROUP FOR THAT TEAM
    MIN((IIF(depth_rank_adp=1,adp_ovr,NULL))) AS adp_ovr_depth1,
    MIN((IIF(depth_rank_adp=2,adp_ovr,NULL))) AS adp_ovr_depth2,
    MIN((IIF(depth_rank_adp=3,adp_ovr,NULL))) AS adp_ovr_depth3,
    -- AGE OF THE PLAYER FOR THE ADP OF 1, 2, 3 ON EACH POSITION GROUP
    MIN((IIF(depth_rank_adp=1,age,NULL))) AS age_depth1,
    MIN((IIF(depth_rank_adp=2,age,NULL))) AS age_depth2,
    MIN((IIF(depth_rank_adp=3,age,NULL))) AS age_depth3,
    -- GAMES PLAYED FOR THE ADP OF 1, 2, 3 ON EACH TEAMS POSITION GROUP
    MIN((IIF(depth_rank_adp=1,g,NULL))) AS g_depth1,
    MIN((IIF(depth_rank_adp=2,g,NULL))) AS g_depth2,
    MIN((IIF(depth_rank_adp=3,g,NULL))) AS g_depth3,
    -- ADP OF THE PLAYERS POSITION GROUP FOR THE 1, 2, 3RD RANKED DEPTH ON EACH TEAM
    MIN((IIF(depth_rank_adp=1,adp_pos_rank,NULL))) AS adp_pos_rank_d1,
    MIN((IIF(depth_rank_adp=2,adp_pos_rank,NULL))) AS adp_pos_rank_d2,
    MIN((IIF(depth_rank_adp=3,adp_pos_rank,NULL))) AS adp_pos_rank_d3,
    -- THE SEASON OF THE PLAYER IN ADP RANK 1, 2, 3
    MIN((IIF(depth_rank_adp=1,yid,NULL))) AS yid_d1,
    MIN((IIF(depth_rank_adp=2,yid,NULL))) AS yid_d2,
    MIN((IIF(depth_rank_adp=3,yid,NULL))) AS yid_d3,
    -- QB STATS
    SUM(pass_cmp) AS tot_pass_cmp,
    SUM(pass_att) AS tot_pass_att,
    SUM(pass_yds) AS tot_pass_yds,
    SUM(pass_td) AS tot_pass_td,
    SUM(pass_int) AS tot_pass_int,
    -- RB STATS
    SUM(rush_att) AS tot_rush_att,
    SUM(rush_yds) AS tot_rush_yds,
    SUM(rush_td) AS tot_rush_td,
    -- WR STATS
    SUM(targets) AS tot_targets,
    SUM(rec) AS tot_rec,
    SUM(rec_yds) AS tot_rec_yds,
    SUM(rec_td) AS tot_rec_td,
    -- OTHER 
    SUM(fumbles) AS tot_fumbles,
    SUM(fumbles_lost) AS tot_fumbles_lost,
    SUM(all_td) AS tot_all_td,
    -- POINTS
    SUM(fantasy_points_ppr) AS tot_fantasy_points_ppr,
    SUM(fppg_ppr) AS tot_fppg_ppr,
    SUM(exp_fppg_ppr) AS tot_exp_fppg_ppr,
    SUM(pick_hype) AS tot_pick_hype,
    SUM(pick_accuracy) AS tot_pick_accuracy,
    -- OTHER
    -- NUMBER OF PLAYERS TRADED
    SUM(IIF(team_change IS NOT NULL,1,0)) AS cnt_team_change
FROM nfl_adp_hype

 * sqlite:////Users/beoconno/Documents/ff/ff_data/raw_data/nfl/src_code/nfl.db
(sqlite3.OperationalError) no such column: fill_team
[SQL: SELECT
    fill_team AS team,
    c_year,
    c_pos,
    -- OVERALL ADP FOR THE PLAYER IN DEPTH 1, 2, 3 OF EACH POSITION GROUP FOR THAT TEAM
    MIN((IIF(depth_rank_adp=1,adp_ovr,NULL))) AS adp_ovr_depth1,
    MIN((IIF(depth_rank_adp=2,adp_ovr,NULL))) AS adp_ovr_depth2,
    MIN((IIF(depth_rank_adp=3,adp_ovr,NULL))) AS adp_ovr_depth3,
    -- AGE OF THE PLAYER FOR THE ADP OF 1, 2, 3 ON EACH POSITION GROUP
    MIN((IIF(depth_rank_adp=1,age,NULL))) AS age_depth1,
    MIN((IIF(depth_rank_adp=2,age,NULL))) AS age_depth2,
    MIN((IIF(depth_rank_adp=3,age,NULL))) AS age_depth3,
    -- GAMES PLAYED FOR THE ADP OF 1, 2, 3 ON EACH TEAMS POSITION GROUP
    MIN((IIF(depth_rank_adp=1,g,NULL))) AS g_depth1,
    MIN((IIF(depth_rank_adp=2,g,NULL))) AS g_depth2,
    MIN((IIF(depth_rank_adp=3,g,NULL))) AS g_depth3,
    -- ADP OF THE PLAYERS POSITION GROUP FOR THE 1, 2